# Melbourne Train Station Analysis

## Importing and Cleaning Data
Data about Melbourne's train stations are compiled from different sources, cleaned, and merged together.
The following data will compiled into a single table and prepared for analysis:
* [Station Location & Number of Routes](#station_location)
* [Station Code](#station_code)
* [Parking Capacity](#parking)
* [Bike Storage Capacity](#bike)
* [Patronage Data](#patron)

In [1]:
import pandas as pd
import numpy as np
import requests as rq
from geopandas import GeoDataFrame as gpd
from bs4 import BeautifulSoup

<a id='station_location'></a>
### Station Location & Number of Routes
Contains Train station names, geographical coordinates, and lines serving the station. These coordinates will be later used to extract nearby establishment and traffic paterns. The lines serving the station will be counted
* Provider: Department of Transport, Victoria, Australia
* Format: GeoJSON
* License: Creative Commons Attribution (CC BY 4.0)
* output file: data/STATION_NAMES.csv

In [2]:
station_df = gpd.from_file('data/raw/PTV_METRO_TRAIN_STATION.mid')
station_df.sample(5)

,STOP_ID,STOP_NAME,LATITUDE,LONGITUDE,TICKETZONE,ROUTES_USING_STOP,geometry
203,20037,Essendon Railway Station (Essendon),-37.756012,144.916198,1,Craigieburn,POINT (144.916198 -37.756012)
140,19965,Coburg Railway Station (Coburg),-37.742345,144.963336,1,Upfield,POINT (144.963336 -37.742345)
3,19828,Crib Point Railway Station (Crib Point),-38.366123,145.204043,2,Stony Point,POINT (145.204043 -38.366123)
201,20035,Strathmore Railway Station (Strathmore),-37.743591,144.927316,1,Craigieburn,POINT (144.927316 -37.743591)
13,19838,East Malvern Railway Station (Malvern East),-37.876930,145.069393,"1,2",Glen Waverley,POINT (145.069393 -37.87693)


In [3]:
station_df_na = station_df.isna()
station_df_na.describe()

,STOP_ID,STOP_NAME,LATITUDE,LONGITUDE,TICKETZONE,ROUTES_USING_STOP,geometry
count,221,221,221,221,221,221,221
unique,1,1,1,1,1,1,1
top,False,False,False,False,False,False,False
freq,221,221,221,221,221,221,221


In [4]:
station_df[['STOP_NAME']] = station_df[['STOP_NAME']].applymap(lambda x: x.split('(')[0].replace(' Railway Station ','').lower())
station_df[['NUMBER_OF_ROUTES']] = station_df[['ROUTES_USING_STOP']].applymap(lambda x: len(x.split(',')))

In [5]:
station_df.drop(columns=['geometry','ROUTES_USING_STOP', 'TICKETZONE', 'STOP_ID'], inplace=True)
station_df.rename(columns={'STOP_NAME':'STATION_NAME'}, inplace=True)

In [6]:
station_df.to_csv('data/STATION_NAMES.csv')
station_df.head()

,STATION_NAME,LATITUDE,LONGITUDE,NUMBER_OF_ROUTES
0,sunbury,-37.579091,144.727319,1
1,diggers rest,-37.627017,144.719922,1
2,stony point,-38.374235,145.221837,1
3,crib point,-38.366123,145.204043,1
4,morradoo,-38.354033,145.189602,1


<a id='station_code'></a>
### Station Codes
Station codes are used by operators to identify different train stations. Some datasets use these codes instead of full station names. The data is scraped from Wikipedia
* Provider: Wikipedia
* Format: Website
* License: Creative Commons Attribution ShareAlike (CC BY-SA 3.0)
* Location: https://en.wikipedia.org/wiki/List_of_Melbourne_railway_stations
* Output file: data/STATION_CODES.csv

In [7]:
wiki_url = 'https://en.wikipedia.org/wiki/List_of_Melbourne_railway_stations'
wiki_content = rq.get(wiki_url)
soup = BeautifulSoup(wiki_content.text, 'html.parser')

In [8]:
data=[]
rows = soup.find('table', attrs={'class':'wikitable sortable'}).find('tbody').find_all('tr')
for row in rows[1:]:
    data.append([row.find_all('td')[0].find('a').get_text().lower(), row.find_all('td')[1].get_text()])
station_code_df = pd.DataFrame(data, columns=['STATION_NAME', 'STATION_CODE'])

In [9]:
station_code_df.isna().describe()

,STATION_NAME,STATION_CODE
count,222,222
unique,1,1
top,False,False
freq,222,222


In [10]:
station_code_df.to_csv('data/STATION_CODES.csv')
station_code_df.head()

,STATION_NAME,STATION_CODE
0,aircraft,ACF
1,alamein,ALM
2,albion,ALB
3,alphington,ALP
4,altona,ALT


<a id='parking'></a>
### Parking Capacity
Data contains the number of free parking spots around train stations. The number is expected to to be large in suburban stations and start decreasing as we get closer to the city. Main stations in the CBD are expected to have no parking spots
* Provider: Department of Transport, Victoria, Australia
* Format: Geojson file
* License: Creative Commons Attribution (CC BY 4.0)
* Output File: data/PARKING_CAPACITY.csv

In [11]:
parking_df = gpd.from_file('data/raw/PTV_TRAIN_CARPARK.mid')
parking_df.sample(5)

,STATION_NAME,COMMUTER_CAPACITY,geometry
248,BMS,20,"POLYGON ((144.920022 -37.681978, 144.919976 -3..."
386,ALB,102,"POLYGON ((144.82443 -37.777556, 144.823984 -37..."
199,SUY,5,"POLYGON ((144.728415 -37.57877, 144.728474 -37..."
161,MLD,6,"POLYGON ((144.961331 -37.756476, 144.961263 -3..."
381,TOT,36,"POLYGON ((144.862502 -37.799503, 144.862573 -3..."


In [12]:
parking_df.drop(columns=['geometry'], inplace=True)
parking_df.rename(columns={'STATION_NAME':'STATION_CODE', 'COMMUTER_CAPACITY':'PARKING_CAPACITY'}, inplace=True)
parking_df.sample(5)

,STATION_CODE,PARKING_CAPACITY
161,MLD,6
213,MLK,17
360,KPL,186
210,CAR,143
400,BBN,32


In [13]:
parking_df.isna().describe()

,STATION_CODE,PARKING_CAPACITY
count,443,443
unique,1,1
top,False,False
freq,443,443


In [14]:
parking_df[['PARKING_CAPACITY']] = parking_df[['PARKING_CAPACITY']].astype('int64')
parking_df = parking_df.groupby('STATION_CODE').sum().reset_index()

In [15]:
parking_df.to_csv('data/PARKING_CAPACITY.csv')
parking_df.head()

,STATION_CODE,PARKING_CAPACITY
0,ACF,201
1,ALB,673
2,ALP,119
3,ASH,252
4,ASP,78


<a id='bike'></a>
###  Bike Storage Capacity

The number of bicycle storage spaces in each station and the type of storage in each station
* Provider: Department of Transport, Victoria, Australia
* Format: Geojson file
* License: Creative Commons Attribution (CC BY 4.0)
* Output File: data/BIKE_CAPACITY.csv

In [16]:
bike_df = gpd.from_file('data/raw/PTV_TRAIN_STATION_BIKE_STORAGE.mid')
bike_df.sample(5)

,STATION_NAME,TYPE,CAPACITY,geometry
676,Huntingdale,Locker,1,POINT (145.102743 -37.911325)
531,Cheltenham,Locker,1,POINT (145.054933 -37.966962)
525,Cheltenham,Locker,1,POINT (145.054887 -37.967258)
724,Mentone,Locker,1,POINT (145.065336 -37.981874)
3,Mount Waverley,Locker,1,POINT (145.127943 -37.87536)


In [17]:
bike_df[['CAPACITY']] = bike_df[['CAPACITY']].replace(['','Unknown'],'0')
bike_df[['CAPACITY']] = bike_df[['CAPACITY']].astype('int64')

In [18]:
bike_df.drop(columns=['geometry'], inplace=True)

In [19]:
bike_df = pd.concat([bike_df,pd.get_dummies(bike_df.TYPE)], axis= 1, sort= False)

In [20]:
bike_df.drop(columns=['TYPE'], inplace= True)

In [21]:
bike_df[['STATION_NAME']] = bike_df[['STATION_NAME']].applymap(lambda x: x.lower())
alt_names = bike_df[['STATION_NAME']].loc[bike_df['STATION_NAME'].str.len() <= 3].applymap(lambda x: station_code_df.loc[station_code_df.STATION_CODE == x.upper(), 'STATION_NAME'].reset_index(drop=True)[0])
bike_df.update(alt_names)

In [22]:
bike_df = bike_df.groupby(by='STATION_NAME').sum().reset_index()

In [23]:
bike_df.rename(columns={'CAPACITY':'BIKE_CAPACITY'}, inplace=True)
bike_df.isna().describe()

,STATION_NAME,BIKE_CAPACITY,Cages,Hoops,Locker,Other,Parkiteer
count,106,106,106,106,106,106,106
unique,1,1,1,1,1,1,1
top,False,False,False,False,False,False,False
freq,106,106,106,106,106,106,106


In [24]:
bike_df.to_csv('data/BIKE_CAPACITY.csv')
bike_df.sample(10)

,STATION_NAME,BIKE_CAPACITY,Cages,Hoops,Locker,Other,Parkiteer
40,glenferrie,8,0,0,8,0,0
82,roxburgh park,25,0,0,0,0,1
6,bentleigh,0,0,0,0,0,1
93,syndal,26,0,0,0,1,1
35,frankston,74,0,1,16,1,2
21,coburg,50,0,2,16,0,1
26,east malvern,32,0,0,6,0,1
96,tottenham,5,0,1,0,0,0
9,boronia,25,0,0,0,0,1
7,berwick,41,0,0,15,0,1


<a id='patron'></a>
### Patronage Data
Includes the average number of passengers boarding each station at different times during weekdays and total riders during weekends. The data also includes the results of a survey about the travel purposes and other modes of transports used during the same journey.  

The data is preprocessed in Excel before being imported into the sheet. 
* Provider: Public Transport Victoria
* Format: Excel sheet
* License: Creative Commons Attribution (CC BY 4.0)

In [25]:
patron_df = pd.read_csv('data/raw/PTV_TRAIN_PATRONAGE.csv')
patron_df.isna().describe()

,STATION_NAME,ANNUAL_PATRONAGE,PRE_PEAK,AM_PEAK,INTERPEAK,PM_PEAK,POST_PEAK,WEEKDAY_ENTRIES,SAT_ENTRIES,SUN_ENTRIES,...,ACCESS_TRAIN,ACCESS_TRAM,ACCESS_WALK,PURPOSE_EDU,PURPOSE_NIL,PURPOSE_APTMNT,PURPOSE_SOCIAL,PURPOSE_TOURIST,PURPOSE_VISIT,PURPOSE_WORK
count,207,207,207,207,207,207,207,207,207,207,...,207,207,207,207,207,207,207,207,207,207
unique,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
top,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
freq,207,207,207,207,207,207,207,207,207,207,...,207,207,207,207,207,207,207,207,207,207


In [26]:
patron_df.sample(5)

,STATION_NAME,ANNUAL_PATRONAGE,PRE_PEAK,AM_PEAK,INTERPEAK,PM_PEAK,POST_PEAK,WEEKDAY_ENTRIES,SAT_ENTRIES,SUN_ENTRIES,...,ACCESS_TRAIN,ACCESS_TRAM,ACCESS_WALK,PURPOSE_EDU,PURPOSE_NIL,PURPOSE_APTMNT,PURPOSE_SOCIAL,PURPOSE_TOURIST,PURPOSE_VISIT,PURPOSE_WORK
51,diggers rest,0.07,42.04,155.43,43.99,14.77,3.77,260,70,50,...,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
113,mckinnon,0.46,103.22,909.46,317.88,235.46,53.98,1620,740,500,...,0.00,0.0,0.71,0.20,0.01,0.03,0.05,0.00,0.07,0.63
47,darebin,0.14,20.21,285.25,99.97,103.11,11.46,520,180,120,...,0.01,0.0,0.48,0.05,0.02,0.05,0.09,0.01,0.31,0.47
103,keon park,0.29,77.27,389.26,229.84,282.95,50.69,1030,480,320,...,0.01,0.0,0.37,0.20,0.00,0.04,0.08,0.00,0.00,0.67
149,patterson,0.21,55.79,340.01,156.48,183.85,23.87,760,360,230,...,0.00,0.0,0.79,0.14,0.00,0.10,0.16,0.00,0.05,0.55


## Merging Data
The above data will be merged together into a single table. The result will be exported as a csv file named 'DATA_FULL.csv'

In [27]:
full_data = pd.merge(station_df, bike_df, on='STATION_NAME', how='outer')

In [28]:
full_data.isna().describe()

,STATION_NAME,LATITUDE,LONGITUDE,NUMBER_OF_ROUTES,BIKE_CAPACITY,Cages,Hoops,Locker,Other,Parkiteer
count,221,221,221,221,221,221,221,221,221,221
unique,1,1,1,1,2,2,2,2,2,2
top,False,False,False,False,True,True,True,True,True,True
freq,221,221,221,221,115,115,115,115,115,115


In [29]:
full_data.fillna(0, inplace=True)
full_data = pd.merge(full_data, station_code_df, on='STATION_NAME', how='outer')

In [30]:
full_data.isna().describe()

,STATION_NAME,LATITUDE,LONGITUDE,NUMBER_OF_ROUTES,BIKE_CAPACITY,Cages,Hoops,Locker,Other,Parkiteer,STATION_CODE
count,223,223,223,223,223,223,223,223,223,223,223
unique,1,2,2,2,2,2,2,2,2,2,2
top,False,False,False,False,False,False,False,False,False,False,False
freq,223,221,221,221,221,221,221,221,221,221,222


In [31]:
full_data.loc[full_data.isna().LONGITUDE == True]

,STATION_NAME,LATITUDE,LONGITUDE,NUMBER_OF_ROUTES,BIKE_CAPACITY,Cages,Hoops,Locker,Other,Parkiteer,STATION_CODE
221,jolimont,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JLI
222,showgrounds,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SGS


In [32]:
full_data.loc[full_data.STATION_NAME == 'jolimont-mcg', 'STATION_CODE'] = 'JLI'

In [33]:
full_data.dropna(inplace=True)

In [34]:
full_data.head()

,STATION_NAME,LATITUDE,LONGITUDE,NUMBER_OF_ROUTES,BIKE_CAPACITY,Cages,Hoops,Locker,Other,Parkiteer,STATION_CODE
0,sunbury,-37.579091,144.727319,1.0,25.0,0.0,0.0,0.0,0.0,1.0,SUY
1,diggers rest,-37.627017,144.719922,1.0,25.0,0.0,0.0,0.0,0.0,1.0,DIT
2,stony point,-38.374235,145.221837,1.0,0.0,0.0,0.0,0.0,0.0,0.0,STY
3,crib point,-38.366123,145.204043,1.0,0.0,0.0,0.0,0.0,0.0,0.0,CPT
4,morradoo,-38.354033,145.189602,1.0,0.0,0.0,0.0,0.0,0.0,0.0,MRO


In [35]:
full_data = pd.merge(full_data, parking_df, on='STATION_CODE', how='outer')

In [36]:
full_data.isna().describe()

,STATION_NAME,LATITUDE,LONGITUDE,NUMBER_OF_ROUTES,BIKE_CAPACITY,Cages,Hoops,Locker,Other,Parkiteer,STATION_CODE,PARKING_CAPACITY
count,226,226,226,226,226,226,226,226,226,226,226,226
unique,2,2,2,2,2,2,2,2,2,2,1,2
top,False,False,False,False,False,False,False,False,False,False,False,False
freq,221,221,221,221,221,221,221,221,221,221,226,165


In [37]:
full_data[['PARKING_CAPACITY']] = full_data[['PARKING_CAPACITY']].fillna(0)

In [38]:
full_data.dropna(inplace = True)
full_data.isna().describe()

,STATION_NAME,LATITUDE,LONGITUDE,NUMBER_OF_ROUTES,BIKE_CAPACITY,Cages,Hoops,Locker,Other,Parkiteer,STATION_CODE,PARKING_CAPACITY
count,221,221,221,221,221,221,221,221,221,221,221,221
unique,1,1,1,1,1,1,1,1,1,1,1,1
top,False,False,False,False,False,False,False,False,False,False,False,False
freq,221,221,221,221,221,221,221,221,221,221,221,221


In [39]:
full_data = pd.merge(full_data, patron_df, on='STATION_NAME', how='outer')

In [40]:
full_data.isna().describe()

,STATION_NAME,LATITUDE,LONGITUDE,NUMBER_OF_ROUTES,BIKE_CAPACITY,Cages,Hoops,Locker,Other,Parkiteer,...,ACCESS_TRAIN,ACCESS_TRAM,ACCESS_WALK,PURPOSE_EDU,PURPOSE_NIL,PURPOSE_APTMNT,PURPOSE_SOCIAL,PURPOSE_TOURIST,PURPOSE_VISIT,PURPOSE_WORK
count,221,221,221,221,221,221,221,221,221,221,...,221,221,221,221,221,221,221,221,221,221
unique,1,1,1,1,1,1,1,1,1,1,...,2,2,2,2,2,2,2,2,2,2
top,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
freq,221,221,221,221,221,221,221,221,221,221,...,207,207,207,207,207,207,207,207,207,207


In [41]:
full_data.loc[full_data.isna().ACCESS_TRAIN == True]

,STATION_NAME,LATITUDE,LONGITUDE,NUMBER_OF_ROUTES,BIKE_CAPACITY,Cages,Hoops,Locker,Other,Parkiteer,...,ACCESS_TRAIN,ACCESS_TRAM,ACCESS_WALK,PURPOSE_EDU,PURPOSE_NIL,PURPOSE_APTMNT,PURPOSE_SOCIAL,PURPOSE_TOURIST,PURPOSE_VISIT,PURPOSE_WORK
2,stony point,-38.374235,145.221837,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,crib point,-38.366123,145.204043,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,morradoo,-38.354033,145.189602,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,bittern,-38.337390,145.178027,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,hastings,-38.305659,145.185980,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,tyabb,-38.259815,145.186401,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,somerville,-38.225342,145.176245,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,baxter,-38.194043,145.160526,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,leawarra,-38.152034,145.139534,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
195,flemington racecourse,-37.787202,144.907589,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
full_data.dropna(inplace=True)

In [43]:
full_data.shape

(207, 36)

In [44]:
full_data.isna().describe()

,STATION_NAME,LATITUDE,LONGITUDE,NUMBER_OF_ROUTES,BIKE_CAPACITY,Cages,Hoops,Locker,Other,Parkiteer,...,ACCESS_TRAIN,ACCESS_TRAM,ACCESS_WALK,PURPOSE_EDU,PURPOSE_NIL,PURPOSE_APTMNT,PURPOSE_SOCIAL,PURPOSE_TOURIST,PURPOSE_VISIT,PURPOSE_WORK
count,207,207,207,207,207,207,207,207,207,207,...,207,207,207,207,207,207,207,207,207,207
unique,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
top,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
freq,207,207,207,207,207,207,207,207,207,207,...,207,207,207,207,207,207,207,207,207,207


In [45]:
full_data.set_index('STATION_CODE', inplace=True)
full_data.to_csv('data/MEL_STATION_DATA.csv')
full_data.head(5)

,STATION_NAME,LATITUDE,LONGITUDE,NUMBER_OF_ROUTES,BIKE_CAPACITY,Cages,Hoops,Locker,Other,Parkiteer,...,ACCESS_TRAIN,ACCESS_TRAM,ACCESS_WALK,PURPOSE_EDU,PURPOSE_NIL,PURPOSE_APTMNT,PURPOSE_SOCIAL,PURPOSE_TOURIST,PURPOSE_VISIT,PURPOSE_WORK
STATION_CODE,,,,,,,,,,,,,,,,,,,,,
SUY,sunbury,-37.579091,144.727319,1.0,25.0,0.0,0.0,0.0,0.0,1.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
DIT,diggers rest,-37.627017,144.719922,1.0,25.0,0.0,0.0,0.0,0.0,1.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
GIR,glen iris,-37.859308,145.058225,1.0,6.0,0.0,0.0,6.0,0.0,0.0,...,0.08,0.06,0.28,0.16,0.01,0.13,0.03,0.07,0.01,0.59
DLG,darling,-37.868957,145.062951,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.47,0.19,0.01,0.03,0.12,0.00,0.04,0.62
EMV,east malvern,-37.876930,145.069393,1.0,32.0,0.0,0.0,6.0,0.0,1.0,...,0.00,0.00,0.16,0.18,0.00,0.02,0.04,0.00,0.06,0.70
